In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [4]:
from tqdm import tqdm

In [5]:
genre_list=['action','adult','adventure','animation','biography','comedy','crime','documentary','family','fantasy','game-show','history','horror','music','musical','mystery','news','reality-tv','romance','sci-fi','short','sport','talk-show','thriller','war','western']

In [6]:
fallback_genre='Unknown'


In [7]:
try:
    with tqdm(total=50,desc='Loading Train data')as pbar:
        train_data =pd.read_csv('Genre Classification Dataset/train_data.txt',sep=':::',header=None,names=['SerialNumber','MOVIE_NAME','GENRE','MOVIE_PLOT'], engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error in loading_train data: {e}")
    

Loading Train data: 100%|██████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 51.58it/s]


In [8]:
X_train = train_data['MOVIE_PLOT'].astype(str).apply(lambda doc:doc.lower())
genre_labels= [genre.split(', ') for genre in train_data["GENRE"]]
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(genre_labels)

In [9]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [10]:
 with tqdm(total=50,desc='Vectorizer data')as pbar:
        X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
        pbar.update(50)

Vectorizer data: 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:11<00:00,  4.26it/s]


In [11]:
 with tqdm(total=50,desc='Training data')as pbar:
        naive_bayes=  MultinomialNB()
        multi_output_classifier = MultiOutputClassifier(naive_bayes)
        multi_output_classifier.fit(X_train_tfidf,y_train)
        pbar.update(50)

Training data: 100%|███████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 33.38it/s]


In [12]:
try:
    with tqdm(total=50,desc='Test Data Loading')as pbar:
        test_data = pd.read_csv('Genre Classification Dataset/test_data.txt',sep=':::',header=None ,names=['SerialNumber',"MOVIE_NAME","MOVIE_PLOT"],engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error Loading Test Data")
    raise

Test Data Loading: 100%|███████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 56.85it/s]


In [13]:
X_test = test_data['MOVIE_PLOT'].astype(str).apply(lambda doc:doc.lower())

In [14]:
with tqdm(total=50,desc='Vectorizing Test Data')as pbar:
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    pbar.update(50)

Vectorizing Test Data: 100%|███████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.79it/s]


In [15]:
with tqdm(total=50,desc='Predicting on Test Data')as pbar:
    y_pred = multi_output_classifier.predict(X_test_tfidf)
    pbar.update(50)

Predicting on Test Data: 100%|█████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 40.51it/s]


In [17]:
test_movie_names = test_data['MOVIE_NAME']
predicted_genre= mlb.inverse_transform(y_pred)
test_results = pd.DataFrame({'MOVIE_NAME':test_movie_names,'PREDICTED_GENRES':predicted_genre})

In [20]:
test_results['PREDICTED_GENRES']= test_results['PREDICTED_GENRES'].apply(lambda genres:[fallback_genre] if len(genres)==0 else genres)

In [50]:
with open('Genre Classification Dataset/test_data_solution.txt','w',encoding ='utf-8') as output_file:
    for _, row in test_results.iterrows():
        print(row["MOVIE_NAME"], row['PREDICTED_GENRES'])
        movie_name = row["MOVIE_NAME"]
        genre_str = ','.join(row['PREDICTED_GENRES'])
        output_file.write(f"{movie_name} ::: {genre_str}\n")


 Edgar's Lunch (1998)  ['Unknown']
 La guerra de papá (1977)  (' drama ',)
 Off the Beaten Track (2010)  (' documentary ',)
 Meu Amigo Hindu (2015)  (' drama ',)
 Er nu zhai (1955)  ['Unknown']
 Riddle Room (2016)  ['Unknown']
 L'amica (1969)  (' drama ',)
 Ina Mina Dika (1989)  ['Unknown']
 Equinox Special: Britain's Tornados (2005)  (' documentary ',)
 Press (2011)  ['Unknown']
 The Firm Man (1975)  ['Unknown']
 Out of Here (2013/I)  ['Unknown']
 Tip 2 (1990)  (' drama ',)
 Hero of the Day (2012)  ['Unknown']
 Das letzte Mal (2000)  ['Unknown']
 The Man in Every Shot (2010)  ['Unknown']
 The Kettles on Old MacDonald's Farm (1957)  (' comedy ',)
 The Patriot (1916)  ['Unknown']
 "The Rogue Legends Series" (2017)  (' documentary ',)
 A Case of Mistaken Identity? (2008)  ['Unknown']
 Dinner for Three (2016)  ['Unknown']
 Crack Baby (????)  ['Unknown']
 Paper Marilyn (1994)  ['Unknown']
 A Tribute to Rick Nelson (1986)  ['Unknown']
 The Silent Rice Fields of Bakan (2016)  (' documentary 

 Mud Man (2016)  ['Unknown']
 Ad-Vice for a Prophet (2005)  ['Unknown']
 "Explaining to an Alien" (2017)  ['Unknown']
 Broken Still One (2017)  (' documentary ',)
 "Sudan's Forgotten Films" (2017)  (' documentary ',)
 Shifting Gaze (2012)  (' documentary ',)
 EuroAmerican (2004)  (' documentary ',)
 "Zona" (2006)  ['Unknown']
 TV One Night Only: Live from the Essence Music Festival (2011)  (' documentary ',)
 Persephone (2008)  ['Unknown']
 Captain Abu Raed (2008)  ['Unknown']
 Chiayi Symphony (2004)  ['Unknown']
 Fatal Deception: Mrs. Lee Harvey Oswald (1993)  ['Unknown']
 A Few Things About Robert Irwin (2017)  (' documentary ',)
 Electric Shadows (2004)  (' documentary ',)
 Shadey (1985)  ['Unknown']
 Cheeks 5: Cop a Feel (1991)  ['Unknown']
 The Ends of the Earth (1982)  (' documentary ',)
 Sortie 234 (1988)  (' comedy ',)
 Unconquering the Last Frontier (2000)  (' documentary ',)
 Dear Walt (2016)  ['Unknown']
 Desamores (2004)  ['Unknown']
 Mädchen, die sich selbst bedienen (1976

 Rwanda Rises Up (2010)  (' documentary ',)
 Barry Hilton: Serial Comic (2011)  ['Unknown']
 5TH NAFCA: African Oscar (2015)  (' documentary ',)
 Moral Kombat (2009)  ['Unknown']
 Unit 4 (1981)  ['Unknown']
 A Different Conversation (2009)  (' documentary ',)
 Belgium Building (2014)  ['Unknown']
 Vertical Living Made Easy (2008)  ['Unknown']
 Chekhov and Maria (2007)  (' drama ',)
 Jon Reep: Metro Jethro (2009)  (' comedy ',)
 Cosmic Slop (1994)  (' documentary ',)
 Történetek a magyar filmröl (1982)  ['Unknown']
 Ernie Banks (????)  ['Unknown']
 Xiong zhai yan zhi (1998)  ['Unknown']
 Horror! (2007)  ['Unknown']
 A Kiss Goodnight (1994)  (' drama ',)
 Feathertop (????)  ['Unknown']
 "Yi tian tu long ji" (2002)  ['Unknown']
 Shark Wars (????)  ['Unknown']
 Viande d'origine française (2009)  (' documentary ',)
 Here There and Everywhere Beatles Fan Film (2018)  (' documentary ',)
 "Wife Swap Australia" (2012)  ['Unknown']
 Popcorn und Paprika (1984)  ['Unknown']
 Sexting in America: Wh

 Ballet for Life (1997)  (' documentary ',)
 Magnificent Burden (2016)  ['Unknown']
 The Keeper of the Keys (2012)  ['Unknown']
 Date Bait (1960)  (' drama ',)
 Mein Vater, die Tunte (2001)  ['Unknown']
 The Lonely Gladiator (2010)  ['Unknown']
 Party Pieces (????)  ['Unknown']
 Coyote Mountain (2010)  ['Unknown']
 Ewir Amora Kelabi (2016)  ['Unknown']
 Green Giants (2015)  (' documentary ',)
 Cartas de Sorolla (2006)  ['Unknown']
 I Melt with You (2011)  ['Unknown']
 "Pashmina Aisha" (2014)  (' drama ',)
 Frankie's Baby (2018)  (' drama ',)
 Making a Scene (2010)  (' documentary ',)
 Spanish for Kids: Adentro y afuera (2009)  (' documentary ',)
 Revolution Now (????)  ['Unknown']
 Cathedral Pines (2006)  ['Unknown']
 Phileas Fortune (2011)  ['Unknown']
 Ahla al awkat (2004)  ['Unknown']
 "Safe at Home" (1986)  ['Unknown']
 30 svidaniy (2016)  ['Unknown']
 Chicagoland Emergency Vehicle Show Volume 3 (2007)  (' documentary ',)
 Live Cinema Workshop (2017)  (' documentary ',)
 Damsels in

 Queen of the Amazons (1947)  ['Unknown']
 Affluenza (2014)  ['Unknown']
 Madame Bovary (1953)  (' drama ',)
 Take It from Me (2001)  (' documentary ',)
 The Poison Rose (????)  ['Unknown']
 Hard XS (2007)  ['Unknown']
 The Cimarron Kid (1952)  ['Unknown']
 Our Hands (1982)  ['Unknown']
 House of Mortal Sin (1976)  (' drama ',)
 La valle dell'eco tonante (1964)  ['Unknown']
 Killergrippe 2008 (2007)  ['Unknown']
 The Adventures of Miss Girl (2007)  ['Unknown']
 Asturias, sus gentes, sus cosas: La mar (1975)  ['Unknown']
 Csend és kiáltás (1968)  ['Unknown']
 The Woods Soldiers (2018)  ['Unknown']
 Bigfoot: Still Tracking a Legend (2015)  (' documentary ',)
 Pizza (2005/I)  ['Unknown']
 The Key (1934)  ['Unknown']
 A Bunny Girl's Tale (1999)  ['Unknown']
 Der starke Ferdinand (1976)  ['Unknown']
 Surreal Soup: A Dream from Last Night (????)  ['Unknown']
 Diamonds and Guns (2008)  ['Unknown']
 Complexion (2008)  ['Unknown']
 A Portrait of the Artist as a Young Man (1977)  ['Unknown']
 Un

...

 Jhinuk Mala (1996)  (' drama ',)
 8 Wheels & Some Soul Brotha' Music (2003)  (' documentary ',)
 Casa privata per le SS (1977)  ['Unknown']
 Na Hannyate (2012)  (' drama ',)
 Violeta de mil colores (2005)  ['Unknown']
 History and Memory: For Akiko and Takashige (1991)  (' documentary ',)
 Pact (2001)  (' drama ',)
 Ulta Seedha (1985)  ['Unknown']
 Rapid Eye Movie (1999)  ['Unknown']
 Thirsty the Movie (2015)  ['Unknown']
 Gerhard Richter: 4 Decades (2005)  (' documentary ',)
 Plainclothes Agents (2011)  (' documentary ',)
 Samaritan (2008/IV)  ['Unknown']
 Amazones 2000 (2000)  (' documentary ',)
 China's Forgotten Daughters (2017)  (' documentary ',)
 Ang Liham sa Tulay ng Usman: The Letter (????)  ['Unknown']
 L'arbitro (1974)  ['Unknown']
 "My OWN Time" (2011)  (' documentary ',)
 Povest o stríbrné jedli (1973)  ['Unknown']
 10/10ths (2017)  (' documentary ',)
 Temp-To-Perm (2017)  ['Unknown']
 The Dionne Quintuplets (1978)  ['Unknown']
 Avatara (????)  ['Unknown']
 Five Hours (20

 Jimmy Johnstone: Lord of the Wing (2004)  ['Unknown']
 Third Party President: Citizen Rocky (2016)  (' documentary ',)
 When Muscles Ruled the World (2002)  (' documentary ',)
 Celine (2017/I)  ['Unknown']
 I Will Not Go Quietly (2010)  ['Unknown']
 Screen Addicts (2008)  ['Unknown']
 Puttin' on the Ritz (1930)  ['Unknown']
 On Culture and Tropics (2014)  (' documentary ',)
 An Inconvenient Tax (2011)  (' documentary ',)
 Distant Vision (2016)  (' documentary ',)
 Reliquias (2011)  ['Unknown']
 Pirate for the Sea (2008)  (' documentary ',)
 No Limbs No Limits (2013)  (' documentary ',)
 Der Brand (2011)  (' drama ',)
 The Blind Productions Project (2005)  ['Unknown']
 Marlene Dietrich: Shadow and Light (1996)  (' documentary ',)
 Don't Worry, We'll Think of a Title (1966)  ['Unknown']
 Ghettokids (2002)  (' drama ',)
 Urban Woodland (2000)  ['Unknown']
 "The Flush" (2005)  ['Unknown']
 Gou hun jiang tou (1976)  ['Unknown']
 Voice of a Nation: My Journey Through Afghanistan (2012)  (' 

 Glass (2001)  (' drama ',)
 Memphis (2013/I)  ['Unknown']
 If Not You (2012)  ['Unknown']
 The Pushcarts leave Eternity Street (2011)  ['Unknown']
 Mi Pogolotti querido (2010)  (' documentary ',)
 Superpacific (1951)  ['Unknown']
 Basic. Holy Spirit (2010)  ['Unknown']
 The Truth (2010/I)  ['Unknown']
 Thrills Spills and Chills (2004)  ['Unknown']
 Maquinas Infernales (2010)  ['Unknown']
 Pain Flower (1996)  ['Unknown']
 "Galgal Hamazal" (1993)  ['Unknown']
 The Adventures of Megabyte Man (2006)  ['Unknown']
 Il grande Blek (1987)  (' drama ',)
 Posse from Hell (1961)  ['Unknown']
 Stanalto Madness Getz: Cool Summer (2002)  (' documentary ',)
 Three's a Charm (????)  ['Unknown']
 The Iron Trail (1921)  ['Unknown']
 The Daughters of Hustle (????)  ['Unknown']
 Tagikaks - Once Were Hunters (2012)  ['Unknown']
 Neverside (2013)  ['Unknown']
 Marching Through Your Head (2010)  ['Unknown']
 El baile de la Victoria (2009)  ['Unknown']
 Losing the West (2013)  (' documentary ',)
 20 Best and

 11. lipnja - Prvi splitski Pride (2011)  (' documentary ',)
 Cada cuatro fotogramas (2009)  (' documentary ',)
 New Beijing: Reinventing a City (2009)  (' documentary ',)
 The Wannabe (????)  ['Unknown']
 Neji-shiki (1998)  (' drama ',)
 Intrusion (2012/I)  ['Unknown']
 Henry Hornbostel in Architecture and Legacy (2013)  (' documentary ',)
 The Fear Factory (2010)  ['Unknown']
 Jon Lord: It's All Music (2013)  ['Unknown']
 Vite! Vite! Je suis en retard (1910)  ['Unknown']
 Hats Off (2008)  ['Unknown']
 Operation Makonaima (1974)  ['Unknown']
 Smartest Girl in Town (1936)  ['Unknown']
 Assignment Auron (2005)  ['Unknown']
 Jex (2014)  ['Unknown']
 In a Small World (2012)  ['Unknown']
 A Perfect Song (2003)  ['Unknown']
 Le cheval 2.1 (2005)  ['Unknown']
 Tianjin Mysteries Perpetrator (2017)  ['Unknown']
 Junkie (2012)  ['Unknown']
 Ragaszkodom a szerelemhez (1943)  ['Unknown']
 View from a Blue Moon (2015)  (' documentary ',)
 Op Zee (2013)  ['Unknown']
 "Banco Nacional de México y su 

 Poveri ma ricchi (2016)  ['Unknown']
 The Rose Laundry (2004)  ['Unknown']
 Cultivate Longevity (2005)  (' documentary ',)
 "Le château des oliviers" (1993)  (' drama ',)
 PhonY (????)  ['Unknown']
 "Make Your Movie" (2004)  ['Unknown']
 Frontier Fury (1943)  ['Unknown']
 The One That Got Away (2015/I)  (' drama ',)
 A szamizdatról (1989)  (' documentary ',)
 No More Joy: The Rise and Fall of New Orleans Movie Theatres (2005)  (' documentary ',)
 High Play (1917)  (' drama ',)
 Taking 'Hostage': Behind the Scenes (2005)  (' documentary ',)
 A Walton Wedding (1995)  (' drama ',)
 The Bus (1965)  (' documentary ',)
 Tillie's Punctured Romance (1928)  ['Unknown']
 "The Bonnie Hunt Show" (1995)  ['Unknown']
 Prisoner of Love: Myuran Sukumaran... (2015)  ['Unknown']
 "Six Windows" (2015)  ['Unknown']
 Quest for Samurai (2008)  ['Unknown']
 I Be Done Been Was Is (1984)  (' documentary ',)
 Rad na odredjeno vreme (1980)  (' drama ',)
 Justice of the Range (1935)  ['Unknown']
 Shadowtown (???

In [46]:
y_train_pred = multi_output_classifier.predict(X_train_tfidf)

In [47]:
accuracy= accuracy_score(y_train,y_train_pred)
recall= recall_score(y_train,y_train_pred,average='micro')
precision= precision_score(y_train,y_train_pred,average='micro')
f1= f1_score(y_train,y_train_pred,average='micro')

In [51]:
with open('Genre Classification Dataset/test_data_solution.txt','a',encoding ='utf-8') as output_file:
    output_file.write("\n\n Model Evaluation Metrics:\n")
    output_file.write(f"Accuracy : {accuracy * 100:.2f}% \n")
    output_file.write(f"\n\n Precision : {precision:.2f}\n")
    output_file.write(f"\n\n Recall: {recall: .2f} \n")
    output_file.write(f"\n\n F1-score: {f1:.2f} \n")
    
print("All the modulation evaluation results and metrics are saved in 'test_data_solution.txt'")

All the modulation evaluation results and metrics are saved in 'test_data_solution.txt'
